In [22]:
import pandas as pd
import numpy as np
import yaml
debug=False
from __future__ import print_function
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [23]:
def compareGOLD(row):
    if debug: global n
#    if row["lexeme"]=="dodo": 
#        debug=True
#    else:
#        debug=False
    if debug and n%250==0: print (n,end=", ")
    if debug: n+=1
    ligne={k:",".join(row[k]) for k in row.keys() if not k in ["lexeme","score","ajouts"]}
#    if debug and n<5: print (ligne)
    lexeme=row["lexeme"]
    score=float(row["score"][0])
    ajouts=int(row["ajouts"][0])
    if not lexeme in conjugaisons:
        conjugaisons[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    if not lexeme in cliquesTypes:
        cliquesTypes[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    identiques=[]
    differents=[]
    missing=[]
    over=[]
    for case in analyseCases:
        if debug:    
            print (lexeme,case)
            print (GOLD[GOLD["lexeme"]==lexeme][case].item())
            print ((GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]))
#        if not case in row:
        if not case in ligne:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("absence",case)
#        elif case in row[row.isnull()] and not GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
        elif ligne[case]=="" and not GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("CLIQUES +NaN",case)
#        elif not case in row[row.isnull()] and GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
        elif ligne[case]!="" and GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
#            over.append(u"%s : %s ≠ Ø" % (case,row[case]))
            over.append(u"%s : %s ≠ Ø" % (case,ligne[case]))
            lexical=False
            if debug: print ("GOLD -NaN",case)            
#        elif (row[case]==GOLD[GOLD["lexeme"]==lexeme][case].item()):
        elif (ligne[case]==GOLD[GOLD["lexeme"]==lexeme][case].item()):
            if debug: print ("egal")
#            identiques.append(u"%s : %s" % (case,row[case]))
            identiques.append(u"%s : %s" % (case,ligne[case]))
            if debug: print ("valeurs id")
#        elif (case in row[row.isnull()]) and (GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]):
        elif ligne[case]=="" and (GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]):
            if debug: print ("NaNs")
            identiques.append(u"%s : %s" % (case,u"ØØØ"))
            if debug: print ("deux NaN")
        else:
            if debug: print ("else")
#            differents.append(u"%s : %s ≠ %s" % (case, row[case],GOLD[GOLD["lexeme"]==lexeme][case].item()))
            differents.append(u"%s : %s ≠ %s" % (case, ligne[case],GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print (u"différence",case)
    if differents:
        conjugaisons[lexeme]["diff"].append(differents)
        cliquesTypes[lexeme]["diff"].append((ligne,score,ajouts))
        if debug: print ("DIFF",len(differents),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug: print (", ".join(differents))
    if missing and over:
        conjugaisons[lexeme]["overmiss"].append(over+missing)
        cliquesTypes[lexeme]["overmiss"].append((ligne,score,ajouts))
        if debug: print ("OVERMISS", len(over), len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug: 
            print (", ".join(missing))
            print (", ".join(over))
    elif missing:
        conjugaisons[lexeme]["miss"].append(missing)
        cliquesTypes[lexeme]["miss"].append((ligne,score,ajouts))
        if debug: print ("MISS", len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug: 
            print (", ".join(missing))
    elif over:
        conjugaisons[lexeme]["over"].append(over)
        cliquesTypes[lexeme]["over"].append((ligne,score,ajouts))
        if debug: print ("OVER", len(over),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug: 
            print (", ".join(over))
    if identiques:
        conjugaisons[lexeme]["ok"].append(identiques)
        cliquesTypes[lexeme]["ok"].append((ligne,score,ajouts))
        if debug: print ("OK", len(identiques), len(identiques)-nombreElements[lexeme],end=saut)
        if debug: print (", ".join(identiques))
#    print ()
    return 

In [24]:
def evaluation(sampleNumber):

    

    if debug: print (", ".join(sampleCases))
    #paradigmesCLIQUES.apply(compareGOLD,axis=1)
    paradigmeSILVER.apply(compareGOLD,axis=1)

    yaml.safe_dump(conjugaisons, file("2015-Data/"+samplePrefix+dateheure()+".yaml",'w'), encoding='utf-8', allow_unicode=True)
    yaml.safe_dump(cliquesTypes, file("2015-Data/"+samplePrefix+"cliques-"+dateheure()+".yaml",'w'), encoding='utf-8', allow_unicode=True)

    precisions={"bon":0,"mauvais":0}
    for verbe in cliquesTypes:
        if debug: print (verbe)
        for cle in cliquesTypes[verbe]:
            if debug: print (cle)
            for element in cliquesTypes[verbe][cle]:
                if debug: print (element[1],element[2])
                if cle in ["ok"]:
                    precisions["bon"]+=element[2]
                elif cle in ["diff"]:
                    precisions["mauvais"]+=element[2]

    print (precisions)
    print ("précision (cliquesTypes)",float(precisions["bon"])/(precisions["bon"]+precisions["mauvais"]))

    basesMesures={"total":0,"bon":0,"mauvais":0}
    for verbe in conjugaisons:
        for cat in conjugaisons[verbe]:
            if conjugaisons[verbe][cat]:
    #            print (verbe,cat,len(conjugaisons[verbe][cat][0]))
                if cat=="miss":
                    basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
                elif cat=="ok":
                    basesMesures["bon"]+=len(conjugaisons[verbe][cat][0])
                    basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
                elif cat=="diff":
                    basesMesures["mauvais"]+=len(conjugaisons[verbe][cat][0])
                    basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
                elif cat=="overmiss":
                    basesMesures["mauvais"]+=len(conjugaisons[verbe][cat][0])
                    basesMesures["total"]+=len(conjugaisons[verbe][cat][0])

    print (basesMesures)
    precisionManip=float(basesMesures["bon"])/(basesMesures["bon"]+basesMesures["mauvais"])
    print ("précision (conjugaison)",precisionManip)
    rappelManip=float(basesMesures["bon"])/(basesMesures["total"])
    print ("rappel (conjugaison)",rappelManip)

In [25]:
sampleNumbers=["01-X"]

for sampleNumber in sampleNumbers:
    goldNumber="00-"+sampleNumber.split("-")[1]
    samplePrefix="MGC-150815-extend-%s-paradigmes"%sampleNumber
    analysisPrefix="MGC-150815-extend-%s"%sampleNumber
    goldPrefix="MGC-150815-extend-%s"%goldNumber

    SILVER=pd.read_csv("2015-Data/"+analysisPrefix+'-Silver.csv',sep=";",encoding="utf8")
    del SILVER["Unnamed: 0"]

    GOLD=pd.read_csv("2015-Data/"+goldPrefix+'-Gold.csv',sep=";",encoding="utf8")
    del GOLD["Unnamed: 0"]

    paradigmeSILVER=SILVER.groupby("lexeme").agg(lambda x: list(set(x.dropna().values))).reset_index()

    sampleCases=paradigmeSILVER.columns.values.tolist()
    for element in [u"lexeme",u"ajouts",u"score"]:
        sampleCases.remove(element)
    analyseCases=sampleCases

    conjugaisons={}
    cliquesTypes={}
    saut="\n"
#    n=0
    evaluation(sampleNumber)


{'bon': 56516, 'mauvais': 5769}
précision (cliquesTypes) 0.907377378181
{'total': 142973, 'bon': 70703, 'mauvais': 7968}
précision (conjugaison) 0.898717443531
rappel (conjugaison) 0.494519944325
